In [1]:
import numpy as np
from matplotlib import pyplot as plt
import scipy as sc
import pandas as pd

In [4]:
# Function to calculate location of L2, under reduced mass constraint
# R = distance between two masses M1, M2
# M1 = Mass of the larger object
# M2 = Mass of the smaller object
# https://en.wikipedia.org/wiki/Lagrange_point#L2
def L2_point(R, M1, M2):
    return R*(M2/(3*M1))**(1/3)

# Defining radius and mass parameters of the system
global M_sun = 1.989e+30
global M_earth = 5.972e+24
global r_sun = 696340e+3    # 696,340 km
global r_earth = 6371e+3    # 6371 km

# Defining constants
global c = 3e+8             # speed of light
global AU = 1.496e+11       # astronomical unit = 149.6 million km
global G = 6.67408e-11      # Gravitational constant
global pc = 3.086e+16       # parsec = 3.26 light years = 206,000 AU = 30.9 trillion km

In [ ]:
# Function to calculate Einstein angle of a lensing system
# M = mass of the lens
# D = Effective lensing distance
def Einstein_angle(M, D):
    return np.sqrt((4*G*M)/(c**2*D))

# Defining lensing system parameters (lengths)
DL =             # Distance between lens and observer
DS =             # Distance between source and observer
DLS =            # Distance between lens and source
D = DL*DS/DLS    # Effective lensing distance
beta =           # Angular position of source
r_S =            # Physical transverse emission size

# Defining lensing system parameters
M = 1e-14        # Mass of lens
z_L = 0          # Redshift of lens (simplifying approximation)

# Derived parameters
theta_E = Einstein_angle(M, D)    # def Einstein_angle(M, D)
r_E = theta_E * DL                # Einstein radius

In [ ]:
# Functions to calculate magnification and brightness flux of each image
# u = beta/theta_E 
    # where beta is the angular position of the source
    # and theta_E is the Einstein angle of the given lensing system
def image_magnification(u):
    return np.abs(0.5 + (u**2 + 2)/(2*u*np.sqrt(u**2 + 4))), np.abs(0.5 - (u**2 + 2)/(2*u*np.sqrt(u**2 + 4)))

# Obtaining parameter values
[mu_plus,mu_minus] = image_magnification(u)
A = mu_plus + mu_minus    # Observed magnification of unresolved images

In [5]:
# Conservative
epsilon = 0.1
nu = 1.2e+20    # 0.5 MeV
NGRB = 10**3
Delta_r = np.array((2*r_earth, L2_point(AU, M_sun, M_earth), 2*AU))    # def L2_point(R, M1, M2)
                                                                       # Delta_r[0] = 2*r_earth, Delta_r[1] = L2_point, Delta_r[2] = 2*AU

In [6]:
# Optimistic
epsilon = 0.01
nu = 1.2e+21    # 5 MeV
NGRB = 10**4
Delta_r = np.array((2*r_earth, L2_point(AU, M_sun, M_earth), 2*AU))    # def L2_point(R, M1, M2)
                                                                       # Delta_r[0] = 2*r_earth, Delta_r[1] = L2_point, Delta_r[2] = 2*AU

In [ ]:
# Function to calculate the upper and lower bound for lens' mass
def Lensing_parallax_conditions(Delta_r, M, r_E, D):
    M_upper = 10e-7 * (Delta_r)**2 * (1/D)
    # WARNING: NOT INCLUDED RADIOMETRIC RESOLUTION CONDITION
    M_lower = np.max((10e-12 * epsilon * (1/D) * (r_S)**2), (0.1*c/nu * c**2/(2*G*(1+z_L))))
    if ((M > M_lower) and (M < M_upper)):
        print(M," is in the given mass range", M_lower, " to ", M_upper)
    else:
        print(M," is not in the given mass range", M_lower, " to ", M_upper)